In [11]:
import os
import pandas as pd
from linux_xbbg import blp

bt_dir = '../adr_results/turnover_lookback/'
adr_info = pd.read_csv('../data/raw/adr_info.csv')
adr_info['adr_ticker'] = adr_info['adr'].str.replace(' US Equity','')
trade_df = pd.read_csv(os.path.join(bt_dir, 'trades.csv'), parse_dates=['timestamp'])
trade_df['date'] = trade_df['timestamp'].dt.strftime('%Y-%m-%d')

In [12]:
p = trade_df.sort_values('timestamp').groupby(['date','ticker'])[['size','price']].first().loc['2025-12-16']

In [13]:
p['ISIN'] = blp.bds((p.index + ' US Equity').to_list(),'ID_ISIN')['value'].values

In [14]:
betas = pd.read_csv('../data/processed/models/ordinary_betas_index_only.csv', index_col=0).loc['2025-12-16'].rename('beta')

In [ ]:
merged = p.merge(betas, left_index=True, right_index=True)
merged = merged.merge(adr_info[['adr_ticker','index_future_bbg','currency']], left_index=True, right_on='adr_ticker')
merged.loc[merged['index_future_bbg']=='Z', 'index_future_bbg'] = 'Z '
merged['index_future_bbg'] += 'H6 Index'
merged['val'] = merged['size'] * merged['price']

fut_price = blp.bdh(merged['index_future_bbg'].unique().tolist(), ['PX_LAST'], '2025-12-16', '2025-12-16').droplevel(1, axis=1).iloc[0]
multiplier = blp.bds(merged['index_future_bbg'].unique().tolist(), ['PRICE_MULTIPLIER']).rename(columns={'value':'PRICE_MULTIPLIER'})
currency = blp.bds(merged['index_future_bbg'].unique().tolist(), ['CURRENCY'])
currency['fx_rate'] = blp.bdh((currency['value'] + 'USD Curncy').tolist(), ['PX_LAST'], '2025-12-16', '2025-12-16').droplevel(1, axis=1).iloc[0].values
fut_merged = fut_price.to_frame('fut_price').merge(multiplier, left_index=True, right_index=True).merge(currency[['fx_rate']], left_index=True, right_index=True)
fut_merged['fut_notional_usd'] = fut_merged['fut_price'] * fut_merged['PRICE_MULTIPLIER'] * fut_merged['fx_rate']

merged = merged.merge(fut_merged[['fut_notional_usd']], left_on='index_future_bbg', right_index=True)
merged['size'] = -(merged['val'] * merged['beta'] / merged['fut_notional_usd'])
futures_pos = merged.groupby('index_future_bbg')[['size']].sum().round()
futures_isin = blp.bds(futures_pos.index.tolist(),'ID_ISIN').rename(columns={'value':'ISIN'})
futures_pos = futures_pos.merge(futures_isin, left_index=True, right_index=True, how='left')
#futures_pos = futures_pos.reset_index(names='bloomberg_symbol')[['bloomberg_symbol','ISIN','size']]

In [40]:
merged.groupby('index_future_bbg')[['size','fut_notional_usd']].agg({'size':'sum', 'fut_notional_usd':'first'}).assign(
    rounded_size = lambda x: x['size'].round()
).assign(
    fut_total_notional = lambda x: x['rounded_size'] * x['fut_notional_usd']
)

,size,fut_notional_usd,rounded_size,fut_total_notional
index_future_bbg,,,,
NHH6 Index,-2.823270,160557.0775,-3.0,-4.816712e+05
VGH6 Index,-58.779633,67533.5030,-59.0,-3.984477e+06
Z H6 Index,-2.340183,130404.4450,-2.0,-2.608089e+05


In [58]:
67533*59

3984447

In [52]:
p[p.index.isin(['EWA','EWU','FEZ','EWJ'])]

,size,price,ISIN
ticker,,,
EWA,-12782,25.795,US4642861037
EWJ,-7526,80.795,US46434G8226
EWU,2639,42.635,US46435G3341
FEZ,-63689,64.405,US78463X2027


600000

In [ ]:
merged.groupby('index_future_bbg')[['size','fut_notional_usd']].sum()


,size,fut_notional_usd
index_future_bbg,,
NHH6 Index,-2.823270,1.123900e+06
VGH6 Index,-58.779633,1.418204e+06
Z H6 Index,-2.340183,2.999302e+06


In [ ]:
merged.groupby('index_future_bbg')[['size']].sum().round()

8      67533.5030
9      67533.5030
10     67533.5030
14     67533.5030
12    130404.4450
15    130404.4450
16    130404.4450
13    130404.4450
7      67533.5030
53     67533.5030
17    130404.4450
18     67533.5030
19     67533.5030
20     67533.5030
21     67533.5030
23    130404.4450
24    130404.4450
3     160557.0775
25    130404.4450
26    130404.4450
27     67533.5030
28    130404.4450
6     160557.0775
4     160557.0775
29    130404.4450
30     67533.5030
32     67533.5030
31     67533.5030
33    130404.4450
34     67533.5030
36    130404.4450
35    130404.4450
37    130404.4450
38    130404.4450
39    130404.4450
42     67533.5030
43     67533.5030
44    130404.4450
5     160557.0775
45    130404.4450
41     67533.5030
1     160557.0775
0     160557.0775
2     160557.0775
46     67533.5030
47     67533.5030
48     67533.5030
49    130404.4450
50    130404.4450
51    130404.4450
52    130404.4450
Name: fut_notional_usd, dtype: float64

In [6]:
equity_portfolio = p.reset_index().rename(columns={'ticker':'bloomberg_symbol'})[['bloomberg_symbol','ISIN','size']]
equity_portfolio['bloomberg_symbol'] += ' US Equity'
equity_future_portfolio = equity_portfolio[~equity_portfolio['bloomberg_symbol'].isin(['EWU US Equity',
                                                                                       'EWJ US Equity',
                                                                                       'EWA US Equity',
                                                                                        'FEZ US Equity',])]
equity_future_portfolio = pd.concat([equity_future_portfolio,
                                    futures_pos], ignore_index=True)
equity_future_portfolio['size'] = equity_future_portfolio['size'].astype(int)

In [162]:
equity_portfolio.to_excel(os.path.join('..','equities_only_portfolio.xlsx'), index=False)
equity_future_portfolio.to_excel(os.path.join('..','futures_and_equities_portfolio.xlsx'), index=False)